In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"..","..","Algorithm encapsulation"))
from SimAnalysis import SimAnalysis
from LMband import LMband


In [2]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [3]:
phase = np.loadtxt("../data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("../data/volume")
qv = np.loadtxt("../data/path_vol")
phase_get = np.load("../data/phase_get.npy")
d = np.loadtxt("../data/distance")
straight_get = np.load("../data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 8, 14
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 1e6
nx = 0.00000001
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","..","speed analysis","下行速度数据.csv"),index_col=0)

In [ ]:
def cycle_analysis():
    #周期灵敏度分析
    for i in [80, 100, 120, 150, 180, 200]:
        lmb=LMband(phase, [i,i], vol, qv, phase_get, d, straight_get, ison, px, pxb, qb,qb_x,cap, lower, upper, 
           linspace, be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],spd_on,spd_in)
        lmb.solve()
        df=lmb.get_dataframe()
        sim=SimAnalysis(df,phase,ex,os.path.join(os.getcwd(),"..","..","vissim","aa","zhongsan.inp"),
                    os.path.join(os.getcwd(),"..","..","vissim","aa","vissim.ini"),
                    os.path.join(os.getcwd(),"..","..","vissim","aa","zhongsan - 副本.inp"))
        sim.start()
        # spdx=sim.analysis_speed(os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.fzp"))
        delayx=sim.analysis_delay(os.path.join(os.getcwd(),"..","..","vissim","aa",'zhongsan - 副本.vlz'))
        tvltx=sim.analysis_traveltime(os.path.join(os.getcwd(),"..","..","vissim","aa",'zhongsan - 副本.rsz'))
        # spdx.to_csv("./sensitivity/speed_cycle{}.csv".format(i),encoding="utf-8-sig")
        delayx.to_csv("./staturation/delay_cycle{}.csv".format(i),encoding="utf-8-sig")
        tvltx.to_csv("./staturation/traveltime_cycle{}.csv".format(i),encoding="utf-8-sig")
cycle_analysis()


In [ ]:
def dwelltime_analysis():
    for i in [10, 12, 14, 16, 18, 20]:
        lmb=LMband(phase, [80,180], vol, qv, phase_get, d, straight_get, ison, px, pxb, qb,qb_x,cap, lower, upper, 
           linspace, be, speedVar[0], speedVar[1],ex, i,lower,upper,speedVar[0], speedVar[1],spd_on,spd_in)
        lmb.solve()
        df=lmb.get_dataframe()
        sim=SimAnalysis(df,phase,ex,os.path.join(os.getcwd(),"..","..","vissim","aa","zhongsan.inp"),
                    os.path.join(os.getcwd(),"..","..","vissim","aa","vissim.ini"),
                    os.path.join(os.getcwd(),"..","..","vissim","aa","zhongsan - 副本.inp"))
        sim.start()
        # spdx=sim.analysis_speed(os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.fzp"))
        delayx=sim.analysis_delay(os.path.join(os.getcwd(),"..","..","vissim","aa",'zhongsan - 副本.vlz'))
        tvltx=sim.analysis_traveltime(os.path.join(os.getcwd(),"..","..","vissim","aa",'zhongsan - 副本.rsz'))
        # spdx.to_csv("./sensitivity/speed_dwt{}.csv".format(i),encoding="utf-8-sig")
        delayx.to_csv("./staturation/delay_dwt{}.csv".format(i),encoding="utf-8-sig")
        tvltx.to_csv("./staturation/traveltime_dwt{}.csv".format(i),encoding="utf-8-sig")

In [4]:
def staturation_analysis():
    for i in [0.4,0.6,0.8, 1, 1.2, 1.4,1.6]:
        lmb=LMband(phase, [80,180], vol*i, qv*i, phase_get, d, straight_get, ison, px, pxb, qb,qb_x,cap, lower, upper, 
           linspace, be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],spd_on,spd_in)
        lmb.solve()
        df=lmb.get_dataframe()
        sim=SimAnalysis(df,phase,ex,os.path.join(os.getcwd(),"..","..","vissim","aa","zhongsan.inp"),
                    os.path.join(os.getcwd(),"..","..","vissim","aa","vissim.ini"),
                    os.path.join(os.getcwd(),"..","..","vissim","aa","zhongsan - 副本.inp"))
        sim.set_saturation(i)
        sim.start()
        # spdx=sim.analysis_speed(os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.fzp"))
        delayx=sim.analysis_delay(os.path.join(os.getcwd(),"..","..","vissim","aa",'zhongsan - 副本.vlz'))
        tvltx=sim.analysis_traveltime(os.path.join(os.getcwd(),"..","..","vissim","aa",'zhongsan - 副本.rsz'))
        # spdx.to_csv("./sensitivity/speed_sta{}.csv".format(i),encoding="utf-8-sig")
        delayx.to_csv("./staturation/delay_sta{}.csv".format(i),encoding="utf-8-sig")
        tvltx.to_csv("./staturation/traveltime_sta{}.csv".format(i),encoding="utf-8-sig")

In [5]:
staturation_analysis()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 2 times.
MIP Presolve eliminated 335 rows and 104 columns.
MIP Presolve modified 996 coefficients.
Aggregator did 52 substitutions.
Reduced MIP has 949 rows, 449 columns, and 3116 nonzeros.
Reduced MIP has 68 binaries, 76 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (3.24 ticks)
Probing fixed 0 vars, tightened 51 bounds.
Probing time = 0.00 sec. (0.32 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 244 coefficients.
Reduced MIP has 949 rows, 449 columns, and 3116 nonzeros.
Reduced MIP has 68 binaries, 76 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.69 ticks)
Probing time = 0.00 sec. (0.30 ticks)
Clique table members: 29.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using 